### 네이버 증권페이지 내의 종목토론실에서 의견 수집

In [1]:
from selenium import webdriver
import time

from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.request import Request
import pandas as pd
import numpy as np

In [2]:
# 크롬 웹브라우저 실행
driver = webdriver.Chrome('./driver/chromedriver')

C:\Users\admin\AppData\Local\Temp/ipykernel_20112/627124562.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./driver/chromedriver')


In [3]:
# 네이버증권 페이지 접속 - 1페이지 테스트
driver.get('https://finance.naver.com/item/board.naver?code=035720&page=1')

In [275]:
# 해당 페이지 소스 가져오기
req = driver.page_source
soup = BeautifulSoup(req, 'html.parser')
soup
# 마지막 페이지 숫자만 추출
last_page = int(soup.select_one('#content > div.section.inner_sub > table:nth-child(3) > tbody > tr > td:nth-child(2) > table > tbody > tr > td.pgRR > a')['href'][35:])
type(last_page) # str -> int로 변경
last_page # 16519

16519

### 테이블로 구성되어있는데, 한페이지 내에서 자식선택자로 모든 tr 가져오기
- content > div.section.inner_sub > table.type2 > tbody > tr:nth-child(3) ~ 3부터 7
- content > div.section.inner_sub > table.type2 > tbody > tr:nth-child(9) ~ 9부터 13
- content > div.section.inner_sub > table.type2 > tbody > tr:nth-child(15) ~ 15부터 19
- content > div.section.inner_sub > table.type2 > tbody > tr:nth-child(21) ~ 21부터 25

- 날짜, 제목, url, 공감수, 비공감수 수집할 것

**공감수와 비공감수**
- " 0 " 인 경우 : ('strong',{'class':'tah p10 gray03'})
- " 0 초과 " 공감 : ('strong',{'class':'tah p10 red01'})
- " 0 초과 " 비공감 : ('strong',{'class':'tah p10 blue01'})


- 날짜에서 2020.01.01까지 조건걸기


- url접속 후 글내용을 위 df에 합치기

In [276]:
# 빈리스트생성
date = []
title = []
url = []
upvote = []
downvote = []


# 네이버증권 페이지 이동
base_url = 'https://finance.naver.com/item/board.naver?code=035720&page='

num = 1

In [277]:

while num <= last_page :
    driver.get('https://finance.naver.com/item/board.naver?code=035720&page='+str(num))

    # 해당 페이지 소스 가져오기
    req = driver.page_source
    soup = BeautifulSoup(req, 'html.parser')

    # 테이블 tr 전체 가져오기 
    for i in range(3,26) : 
        boxes = soup.select('#content > div.section.inner_sub > table.type2 > tbody > tr:nth-child('+str(i)+')')
    #         print(boxes)

        # tr 코드 하나씩 for문으로 읽기
        for box in boxes :

            # 중간에 구분선 tr이 있어서 에러발생. 예외처리
            try :
                # 2022~2020 년도 작성글만 수집
                year_check = box.find('span',{'class':'tah p10 gray03'}).text[:-6]

                if "2022"or"2021"or"2020" in year_check : 

                    # 날짜
                    date.append(year_check)
                    # 제목
                    title.append(box.find('a')['title'])
                    # url
                    url.append(box.find('a')['href'])
                    # 공감
                    upvote.append(box.select('strong')[0].text)
                    # 비공감
                    downvote.append(box.select('strong')[1].text)

            except :
                continue

    # VIEW탭에서 수집한 url 데이터프레임
    discuss_df = pd.DataFrame({'작성일':date,'제목':title,'url':url, '공감':upvote, '비공감':downvote})
    
    if num%5 == 0 :
        discuss_df.to_csv('./crawl_data/discuss_df_df.csv')
    
    num+=1

In [279]:
driver.close()

### 페이지 넘기기
- url : https://finance.naver.com/item/board.naver?code=035720&page= "페이지번호"

1. select나 xpath를 통해 드라이버로 클릭?

2. for문으로 마지막페이지까지..?

### 드라이버로 페이지 이동 후 데이터 수집

- 1페이지 
- //*[@id="content"]/div[2]/table[2]/tbody/tr/td[2]/table/tbody/tr/td[1]/a
- //*[@id="content"]/div[2]/table[2]/tbody/tr/td[2]/table/tbody/tr/td[2]/a
- //*[@id="content"]/div[2]/table[2]/tbody/tr/td[2]/table/tbody/tr/td[3]/a
- ...
- //*[@id="content"]/div[2]/table[2]/tbody/tr/td[2]/table/tbody/tr/td[10]/a
- 1페이지 > 다음
- //*[@id="content"]/div[2]/table[2]/tbody/tr/td[2]/table/tbody/tr/td[11]/a


- 2페이지
- //*[@id="content"]/div[2]/table[2]/tbody/tr/td[2]/table/tbody/tr/td[3]/a
- //*[@id="content"]/div[2]/table[2]/tbody/tr/td[2]/table/tbody/tr/td[3]/a
- //*[@id="content"]/div[2]/table[2]/tbody/tr/td[2]/table/tbody/tr/td[4]/a
- ...
- //*[@id="content"]/div[2]/table[2]/tbody/tr/td[2]/table/tbody/tr/td[12]/a
- //*[@id="content"]/div[2]/table[2]/tbody/tr/td[2]/table/tbody/tr/td[12]/a
- 2페이지 > 다음
- //*[@id="content"]/div[2]/table[2]/tbody/tr/td[2]/table/tbody/tr/td[13]/a
- //*[@id="content"]/div[2]/table[2]/tbody/tr/td[2]/table/tbody/tr/td[13]/a 

In [ ]:
from selenium import webdriver
import time

from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.request import Request
import pandas as pd
import numpy as np

In [232]:
# 크롬 웹브라우저 실행
driver = webdriver.Chrome('./driver/chromedriver')

# 카카오종목토론실 1페이지로 일단 접속
driver.get('https://finance.naver.com/item/board.naver?code=035720&page=1')

# 1페이지 소스 가져오기
req = driver.page_source
soup = BeautifulSoup(req, 'html.parser')

# 네이버증권 페이지 접속 - 1페이지 테스트
base_url = 'https://finance.naver.com/item/board.naver?code=035720&page='
# 마지막 페이지 숫자만 추출
last_page = int(soup.select_one('#content > div.section.inner_sub > table:nth-child(3) > tbody > tr > td:nth-child(2) > table > tbody > tr > td.pgRR > a')['href'][35:])
type(last_page) # str -> int로 변경
last_page # 16519

C:\Users\admin\AppData\Local\Temp/ipykernel_22500/2879495366.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./driver/chromedriver')


16519

In [ ]:
# 네이버증권 페이지 접속 - 1페이지 테스트
#driver.get('https://finance.naver.com/item/board.naver?code=035720&page=1')
#content > div.section.inner_sub > table:nth-child(3) > tbody > tr > td:nth-child(2) > table > tbody > tr > td.on > a
#content > div.section.inner_sub > table:nth-child(3) > tbody > tr > td:nth-child(2) > table > tbody > tr > td:nth-child(2) > a
#content > div.section.inner_sub > table:nth-child(3) > tbody > tr > td:nth-child(2) > table > tbody > tr > td:nth-child(5) > a
#content > div.section.inner_sub > table:nth-child(3) > tbody > tr > td:nth-child(2) > table > tbody > tr > td:nth-child(10) > a

In [235]:
# 네이버증권 페이지 접속 - 1페이지 테스트
base_url = 'https://finance.naver.com/item/board.naver?code=035720&page='

for i in range(1,11) :
    url = base_url + str(i)
    driver.get(url)

# 다음 페이지 클릭 - 11페이지로 넘어간다.
driver.find_element_by_xpath('//*[@id="content"]/div[2]/table[2]/tbody/tr/td[2]/table/tbody/tr/td[12]/a').click()

C:\Users\admin\AppData\Local\Temp/ipykernel_22500/2101149385.py:9: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="content"]/div[2]/table[2]/tbody/tr/td[2]/table/tbody/tr/td[12]/a').click()


In [237]:
for i in range(11,21) :
    url = base_url + str(i)
    driver.get(url)
    
# 11~20페이지, 다음버튼까지 클릭 - 21페이지
driver.find_element_by_xpath('//*[@id="content"]/div[2]/table[2]/tbody/tr/td[2]/table/tbody/tr/td['+str(i)+']/a').click()
time.sleep(0.5)

C:\Users\admin\AppData\Local\Temp/ipykernel_22500/330087774.py:6: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="content"]/div[2]/table[2]/tbody/tr/td[2]/table/tbody/tr/td['+str(i)+']/a').click()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="content"]/div[2]/table[2]/tbody/tr/td[2]/table/tbody/tr/td[20]/a"}
  (Session info: chrome=100.0.4896.88)
Stacktrace:
Backtrace:
	Ordinal0 [0x00429943+2595139]
	Ordinal0 [0x003BC9F1+2148849]
	Ordinal0 [0x002B4528+1066280]
	Ordinal0 [0x002E0FD4+1249236]
	Ordinal0 [0x002E11CB+1249739]
	Ordinal0 [0x0030D812+1431570]
	Ordinal0 [0x002FBA34+1358388]
	Ordinal0 [0x0030BAF2+1424114]
	Ordinal0 [0x002FB806+1357830]
	Ordinal0 [0x002D6086+1204358]
	Ordinal0 [0x002D6F96+1208214]
	GetHandleVerifier [0x005CB232+1658114]
	GetHandleVerifier [0x0068312C+2411516]
	GetHandleVerifier [0x004BF261+560433]
	GetHandleVerifier [0x004BE366+556598]
	Ordinal0 [0x003C286B+2173035]
	Ordinal0 [0x003C75F8+2192888]
	Ordinal0 [0x003C76E5+2193125]
	Ordinal0 [0x003D11FC+2232828]
	BaseThreadInitThunk [0x755CFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77747A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77747A

In [278]:
driver.current_url

'https://finance.naver.com/item/board.naver?code=035720&page=16519'

### 테스트데이터 수집 (2022.04.18~04.27)

In [4]:
from selenium import webdriver
import time

from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.request import Request
import pandas as pd
import numpy as np

In [5]:
# 크롬 웹브라우저 실행
driver = webdriver.Chrome('./driver/chromedriver')

C:\Users\admin\AppData\Local\Temp/ipykernel_20112/627124562.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./driver/chromedriver')


In [9]:
# 빈리스트생성
date = []
title = []
url = []

# 네이버증권 페이지 이동 
base_url = 'https://finance.naver.com/item/board.naver?code=035720&page='
# 84페이지까지
last_page = 84

num = 1

In [10]:

while num <= last_page :
    driver.get('https://finance.naver.com/item/board.naver?code=035720&page='+str(num))

    # 해당 페이지 소스 가져오기
    req = driver.page_source
    soup = BeautifulSoup(req, 'html.parser')

    # 테이블 tr 전체 가져오기 
    for i in range(3,26) : 
        boxes = soup.select('#content > div.section.inner_sub > table.type2 > tbody > tr:nth-child('+str(i)+')')
    #         print(boxes)

        # tr 코드 하나씩 for문으로 읽기
        for box in boxes :

            # 중간에 구분선 tr이 있어서 에러발생. 예외처리
            try :
                # 2022~2020 년도 작성글만 수집
                year_check = box.find('span',{'class':'tah p10 gray03'}).text[:-6]

                if "2022.04.17" not in year_check : 

                    # 날짜
                    date.append(year_check)
                    # 제목
                    title.append(box.find('a')['title'])
                    # url
                    url.append(box.find('a')['href'])

            except :
                continue

    # VIEW탭에서 수집한 url 데이터프레임
    discuss_df = pd.DataFrame({'작성일':date,'제목':title,'url':url})
    
    if num%5 == 0 :
        discuss_df.to_csv('./crawl_data/test_discuss.csv')
    
    num+=1

In [13]:
discuss_df_1 = discuss_df.drop(1667)
discuss_df_1
discuss_df_1.to_csv('./crawl_data/test_discuss.csv')

In [14]:
driver.close()